<a href="https://colab.research.google.com/github/mrasw/Big-Data/blob/main/Assignment_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# sample_movielens_ratings.txt

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.3.0/spark-3.3.0-bin-hadoop2.tgz   
!tar xf spark-3.3.0-bin-hadoop2.tgz   
!pip install -q findspark

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop2"

!export  | grep SPARK_HOME

declare -x SPARK_HOME="/content/spark-3.3.0-bin-hadoop2"


In [6]:
!pip install -U --upgrade pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 22.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=21d11be0f3f4668052b261721d16b632ea5760c379cd37697c3dc4d827a07226
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [7]:
import pyspark

print(pyspark.__version__)

!export  | grep SPARK_HOME

3.3.2
declare -x SPARK_HOME="/content/spark-3.3.0-bin-hadoop2"


In [8]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
import pyspark.pandas as ps
import pandas as pd

/usr/local/lib/python3.9/dist-packages/pyspark/pandas/__init__.py:49: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [9]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

# spark = SparkSession.builder.appName("coba").getOrCreate()

In [23]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [25]:
lines = spark.read.text("/content/gdrive/My Drive/sample_movielens_ratings.txt").rdd
parts = lines.map(lambda row: row.value.split("::"))
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=int(p[2]), timestamp=int(p[3])))

# print(ratingsRDD)

ratings = spark.createDataFrame(ratingsRDD)
(training, test) = ratings.randomSplit([0.8, 0.2])

In [27]:
ratings

userId,movieId,rating,timestamp
0,2,3,1424380312
0,3,1,1424380312
0,5,2,1424380312
0,9,4,1424380312
0,11,1,1424380312
0,12,2,1424380312
0,15,1,1424380312
0,17,1,1424380312
0,19,1,1424380312
0,21,1,1424380312


#RegParam 0.1

##Max iter 5

In [33]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.1, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

In [34]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

In [35]:
rmse

0.9746284416563535

##Max iter 10




In [36]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=10, regParam=0.1, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

In [37]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

In [38]:
rmse

0.9359626191567274

##Max iter 20




In [39]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=20, regParam=0.1, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

In [40]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

In [41]:
rmse

0.9244056059249371

#RegParam 0.5

##Max iter 5

In [42]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.5, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

In [43]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

In [44]:
rmse

1.1659304109444515

##Max iter 10




In [45]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=10, regParam=0.5, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

In [46]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

In [47]:
rmse

1.164917113496743

##Max iter 20




In [48]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=20, regParam=0.5, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

In [49]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

In [50]:
rmse

1.164730490367786

#RegParam 1.0

##Max iter 5

In [51]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=1.0, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

In [52]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

In [53]:
rmse

1.4416931739411618

##Max iter 10




In [54]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=10, regParam=1.0, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

In [55]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

In [56]:
rmse

1.4417034243200963

##Max iter 20




In [57]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=20, regParam=1.0, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

In [58]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

In [59]:
rmse

1.4417034497178616

# Conclutions

> The Higher RegParam value, more Higher Root Mean Square Error (RMSE)


> If RegParam not 1.0, more Higher MaxIter value, will return better Root Mean Square Error (RMSE)





#Generate top 10 of each candidate

In [31]:
# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)

# Generate top 10 movie recommendations for a specified set of users
users = ratings.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 10)
# Generate top 10 user recommendations for a specified set of movies
movies = ratings.select(als.getItemCol()).distinct().limit(3)
movieSubSetRecs = model.recommendForItemSubset(movies, 10)

In [32]:
print(userRecs,users,userSubsetRecs,movieRecs,movieSubSetRecs,movies)

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    20|[{22, 4.8611417},...|
|    10|[{46, 4.3125963},...|
|     0|[{92, 3.8451357},...|
|     1|[{62, 3.5393393},...|
|    21|[{53, 5.0688987},...|
|    11|[{46, 8.23523}, {...|
|    12|[{27, 5.0697393},...|
|    22|[{74, 5.1679797},...|
|     2|[{8, 5.128191}, {...|
|    13|[{93, 3.8468683},...|
|     3|[{51, 5.0006504},...|
|    23|[{83, 5.3734436},...|
|     4|[{51, 4.797965}, ...|
|    24|[{18, 5.394491}, ...|
|    14|[{52, 5.3739014},...|
|     5|[{55, 4.80079}, {...|
|    15|[{46, 4.740991}, ...|
|    25|[{65, 5.4152074},...|
|    26|[{46, 8.457689}, ...|
|     6|[{93, 4.1779847},...|
+------+--------------------+
only showing top 20 rows
 +------+
|userId|
+------+
|    26|
|    29|
|    19|
+------+
 +------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    26|[{46, 8.457689}, ...|
|    19|[{90, 3.8550153},...|
|    29|[{46, 4.7204943},...|
+------+--